In [3]:
!pip install cv2

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [2]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.llms import Ollama  # Use Ollama as LLM


In [5]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\ankit\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.


True

In [3]:
local_path = "sample.pdf"

# Local PDF file uploads
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
else:
    print("Upload a PDF file")

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, 
                                                  chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [5]:
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
    collection_name="local-ai",
)

OllamaEmbeddings: 100%|████████████████████████████████████████████████████████████████| 22/22 [01:15<00:00,  3.45s/it]


In [6]:
local_model = "llama3"
llm = ChatOllama(model=local_model)

In [7]:
# QUERY_PROMPT = PromptTemplate(
#     input_variables=["question"],
#     template="""You are an financial ai assistant your work is to answer from the documment related to finance""",
# )
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Answer the following question using ONLY the provided financial data.  
If the answer is not present in the data, respond with "The required information is not available."  
Do NOT make assumptions:{question}  
"""
)

In [8]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), llm, prompt=QUERY_PROMPT
)

In [9]:
# template = """Answer the question based ONLY on the following context:
# {context}
# Question: {question}
# """

# prompt = ChatPromptTemplate.from_template(template)
template = """Answer the question based ONLY on the following financial data:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)



In [10]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# chain = (
#     {"question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

In [11]:
def retrieve_context(question):
    retrieved_docs = retriever.invoke(question)  # Retrieve documents
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])  # Combine text
    return {"context": context, "question": question}

In [13]:
# chain.invoke(input("What is the document all about?"))
chain.invoke(retrieve_context(input()))


  What was Infosys' total equity as of March 31, 2024


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.02s/it]


"I'm only going to answer the question based on the provided financial data. Unfortunately, there is no mention of Infosys' total equity as of March 31, 2024 in the given document. The provided data includes information about contingent liabilities and commitments, but not total equity. Therefore, I cannot provide an answer to this question based on the provided data."

In [16]:
import streamlit as st

ModuleNotFoundError: No module named 'streamlit'

In [46]:
import pdfplumber
import pandas as pd

pdf_path = "sample.pdf"

with pdfplumber.open(pdf_path) as pdf:
    tables = []
    for page in pdf.pages:
        extracted_tables = page.extract_tables()
        for table in extracted_tables:
            df = pd.DataFrame(table)
            tables.append(df)

# Display extracted tables
for i, df in enumerate(tables):
    print(f"Table {i+1}:")
    print(df)


Table 1:
                        0 1
0  Deferred Contract Cost  
Table 2:
                                                   0       1        2        3
0                                               None  35,199  145,285  137,575
1  Revenue from products and platforms 1,859 2,24...    None     None     None
Table 3:
      0     1     2
0  None        None
1        None  None
2  None  None      
3        None  None
4  None  None      
5        None  None
Table 4:
                                                   0
0                                                   
1  Particulars Category of ROU asset\nLand Buildi...
2   Balance as of January 1, 2024 607 3,527 18 2,740
3                          Additions* — 61 2 376 439
4                                          Deletions
5                              Impairment# — — — — —
6                                       Depreciation
7        Translation difference — (13) (1) (35) (49)
8  Balance as of March 31, 2024 605 3,298 17 2,63...
T

In [54]:
# import camelot
import camelot.io as camelot
import pandas as pd
tables = camelot.read_pdf(pdf_path, pages='all')
for i, table in enumerate(tables):
    print(f"Table {i+1}:")
    print(table.df)

In [56]:
tables


<TableList n=0>

In [58]:
from docx import Document
import pandas as pd

def extract_text_and_tables(doc_path):
    doc = Document(doc_path)
    text_data = []
    table_data = []

    for para in doc.paragraphs:
        text_data.append(para.text)

    for table in doc.tables:
        table_content = []
        for row in table.rows:
            table_content.append([cell.text.strip() for cell in row.cells])
        table_data.append(pd.DataFrame(table_content))  # Store as DataFrame

    return "\n".join(text_data), table_data  # Text and tables separately

# Example usage
text, tables = extract_text_and_tables("sample.docx")
for i, table in enumerate(tables):
    print(f"Table {i+1}:\n", table)


Table 1:
           0
0  Page No.
1         1
2         2
3         3
4         5
Table 2:
                                                    0
0                                      (In ₹ crore )
1  Condensed Consolidated Balance Sheets as at\tN...
Table 3:
                                     0       1       2
0  Property, plant and equipment\t2.2  12,370  13,346
1           Right-of-use assets\t2.19   6,552   6,882
2            Capital work-in-progress     293     288
3                       Goodwill\t2.3   7,303   7,248
4             Other intangible assets   1,397   1,749
Table 4:
                            0                          1       2       3
0                Investments                        2.4  11,708  12,569
1                      Loans                        2.5      34      39
2     Other financial assets                        2.6   3,105   2,798
3  Deferred tax assets (net)  Deferred tax assets (net)     454   1,245
4    Income tax assets (net)    Income tax as

In [59]:
from docx import Document
import pandas as pd
import os

def extract_tables_from_docx(doc_path, output_folder="tables"):
    """Extract tables from a Word document and save them as CSV files."""
    doc = Document(doc_path)
    table_data = []

    # Create output folder if not exists
    os.makedirs(output_folder, exist_ok=True)

    for i, table in enumerate(doc.tables):
        table_content = []
        
        # Extract rows
        for row in table.rows:
            table_content.append([cell.text.strip() for cell in row.cells])

        # Convert to DataFrame
        df = pd.DataFrame(table_content)

        # Save as CSV
        csv_filename = os.path.join(output_folder, f"table_{i+1}.csv")
        df.to_csv(csv_filename, index=False, header=False)
        table_data.append(df)

        print(f"Saved Table {i+1} as {csv_filename}")

    return table_data

# Example usage
tables = extract_tables_from_docx("sample.docx")

# Display extracted tables
for i, table in enumerate(tables):
    print(f"\nTable {i+1}:\n", table)


Saved Table 1 as tables\table_1.csv
Saved Table 2 as tables\table_2.csv
Saved Table 3 as tables\table_3.csv
Saved Table 4 as tables\table_4.csv
Saved Table 5 as tables\table_5.csv
Saved Table 6 as tables\table_6.csv
Saved Table 7 as tables\table_7.csv
Saved Table 8 as tables\table_8.csv
Saved Table 9 as tables\table_9.csv
Saved Table 10 as tables\table_10.csv
Saved Table 11 as tables\table_11.csv
Saved Table 12 as tables\table_12.csv
Saved Table 13 as tables\table_13.csv
Saved Table 14 as tables\table_14.csv
Saved Table 15 as tables\table_15.csv
Saved Table 16 as tables\table_16.csv
Saved Table 17 as tables\table_17.csv
Saved Table 18 as tables\table_18.csv
Saved Table 19 as tables\table_19.csv
Saved Table 20 as tables\table_20.csv
Saved Table 21 as tables\table_21.csv
Saved Table 22 as tables\table_22.csv
Saved Table 23 as tables\table_23.csv
Saved Table 24 as tables\table_24.csv
Saved Table 25 as tables\table_25.csv
Saved Table 26 as tables\table_26.csv
Saved Table 27 as tables\table

In [3]:
from docx import Document
import pandas as pd

def extract_text_and_tables(doc_path):
    doc = Document(doc_path)
    text_data = []
    table_data = []

    for para in doc.paragraphs:
        text_data.append(para.text)

    for table in doc.tables:
        table_content = []
        for row in table.rows:
            table_content.append([cell.text.strip() for cell in row.cells])
        table_data.append(pd.DataFrame(table_content))  # Store as DataFrame

    return "\n".join(text_data), table_data  # Text and tables separately

# Example usage
text, tables = extract_text_and_tables("sample.docx")
for i, table in enumerate(tables):
    print(f"Table {i+1}:\n", table)


Table 1:
           0
0  Page No.
1         1
2         2
3         3
4         5
Table 2:
                                                    0
0                                      (In ₹ crore )
1  Condensed Consolidated Balance Sheets as at\tN...
Table 3:
                                     0       1       2
0  Property, plant and equipment\t2.2  12,370  13,346
1           Right-of-use assets\t2.19   6,552   6,882
2            Capital work-in-progress     293     288
3                       Goodwill\t2.3   7,303   7,248
4             Other intangible assets   1,397   1,749
Table 4:
                            0                          1       2       3
0                Investments                        2.4  11,708  12,569
1                      Loans                        2.5      34      39
2     Other financial assets                        2.6   3,105   2,798
3  Deferred tax assets (net)  Deferred tax assets (net)     454   1,245
4    Income tax assets (net)    Income tax as

In [4]:
def format_table_as_markdown(table_df):
    """Convert a Pandas DataFrame to a Markdown table for embedding."""
    return table_df.to_markdown(index=False)

# Convert tables to structured text
formatted_tables = [format_table_as_markdown(tbl) for tbl in tables]


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

chroma_db = Chroma(persist_directory="db", embedding_function=OpenAIEmbeddings())

# Store text
chroma_db.add_texts([text])

# Store tables
for formatted_table in formatted_tables:
    chroma_db.add_texts([formatted_table])


In [22]:
chroma_db = Chroma(persist_directory="db", embedding_function=OllamaEmbeddings(model="nomic-embed-text"))  


In [23]:
chroma_db.add_texts([text])


['210bac11-9b3f-4e25-872f-0a488e19eeea']

In [24]:

# Store formatted tables
for formatted_table in formatted_tables:
    chroma_db.add_texts([formatted_table])

In [25]:
def format_docs(docs):
    """Extract and format retrieved documents."""
    return "\n\n".join([doc.page_content for doc in docs])


In [33]:
QUERY_PROMPT = PromptTemplate.from_template("Answer the following question based on the provided context:\n\n{context}\n\nQuestion: {question}")

retriever = chroma_db.as_retriever()

In [34]:
llm = Ollama(model="llama3")

In [40]:
chain = (
    {"context": retriever | RunnableLambda(format_docs), "question": RunnablePassthrough()}
    | QUERY_PROMPT
    | llm
)

# Ask a question
question = "What was Infosys' total equity as of March 31, 2024"
# response = chain.invoke(question)
# print(response)
response = chain.invoke({"context": "Some retrieved documents", "question": question})
print(response)

The context does not provide information about Infosys' total equity as of March 31, 2024. The provided context only shows the changes in the carrying value of right-of-use assets for the three months ended March 31, 2024 and 2023, which are related to assets such as land, buildings, vehicles, and computers. It does not provide any information about Infosys' total equity.
